<a href="https://colab.research.google.com/github/dernameistegal/airbnb_price/blob/main/data_utils/munich/SavingDataInDrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This simple introduction to the Airbnb data set(s) will give you a short overview over the available data. The city used for this introduction is Berlin, hence if you want to run the exact same notebook for a different city you would need to change a few minor details. Otherwise, if you have downloaded all necessary data sets and run this notebook in the same directory it should run smoothly.

### Index
1. Load data set
2. Price analysis
    * (Inspect reviews)
3. Main file (listings.csv.gz)
4. "Analyze" Images
5. "Analyze" Reviews
6. Calendar file
7.  neighbourhoods Geo.json file

In [14]:
#@title imports
%%capture
!pip install transformers
!pip install geopandas
import json
import os
import math
import pandas as pd
import gzip
from PIL import Image
import matplotlib.pyplot as plt
import descartes
import geopandas as gpd
import requests
from io import BytesIO
import matplotlib.image as mpimg
from tqdm import tqdm
from PIL import Image
import numpy as np
from tqdm import tqdm

from shapely.geometry import Point, Polygon

import seaborn as sns

from transformers import pipeline

import folium
from folium.plugins import FastMarkerCluster
from branca.colormap import LinearColormap

In [2]:
#@title mount drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [24]:
# make directories in drive
!mkdir -p /content/drive/MyDrive/Colab/airbnb/munich/hostpics/hostpics_raw
!mkdir -p /content/drive/MyDrive/Colab/airbnb/munich/thumbnails/thumbnails_raw

In [25]:
#read files and show header for overview
listings_meta = pd.read_pickle("/content/drive/MyDrive/Colab/airbnb/munich/listings.pickle")

# Save Images (only run once)

In [26]:
# descriptive statistics for availability of pictures
n_no_hostpic = sum(listings_meta["host_picture_url"].isnull()) 
n_no_thumbnail = sum(listings_meta["picture_url"].isnull())
n_hosts_no_thumbnail = len(np.unique(listings_meta["host_id"][listings_meta["host_picture_url"].isnull()]))
print(f"{n_no_hostpic} listings have no hostpic. In total, {n_hosts_no_thumbnail} hosts have no hostpic. {n_no_thumbnail} listings have no thumbnail.")

0 listings have no hostpic. In total, 0 hosts have no hostpic. 0 listings have no thumbnail.


In [27]:
IMAGE_SIZE = [224, 224]

In [28]:
# instantiate list of all ids where url does not work
pic_malfunction = []

# save hostpics that are available
for i in tqdm(range(len(listings_meta))):

    # get url
    url = listings_meta.iloc[i]["host_picture_url"]

    # check if url is not available
    if pd.isna(url):
        continue

    # scrape url
    response = requests.get(url)

    # check if url does not work
    try:
        img_plot = Image.open(BytesIO(response.content)).resize(IMAGE_SIZE)
    except:
        pic_malfunction.append(listings_meta.iloc[i]["id"])
        continue
    
    # save rgb data
    rgb_data = np.array(img_plot)
    save_path = "/content/drive/MyDrive/Colab/airbnb/munich/hostpics/hostpics_raw/hostpic" + str(listings_meta.iloc[i]["id"])
    np.save(save_path, rgb_data)

100%|██████████| 999/999 [01:17<00:00, 12.94it/s]


In [32]:
# save ids where host pics are not available (either no url or non-functioning url) in dictionary
nopic = np.unique(listings_meta["id"][listings_meta["host_picture_url"].isnull()])
nopic = list(nopic)
indices = nopic + pic_malfunction

missing_data = {"hostpic": [int(ind) for ind in indices]}

temp_file = open("/content/drive/MyDrive/Colab/airbnb/munich/missing_data.json", "w")
json.dump(missing_data, temp_file)
temp_file.close()

In [37]:
# instantiate list of all ids where url does not work
pic_malfunction = []

# save thumbnails that are available
for i in range(len(listings_meta)):

    # get url
    url = listings_meta.iloc[i]["picture_url"]

    # check if url is not available
    if pd.isna(url):
        continue

    # scrape url
    response = requests.get(url)

    # check if url does not work
    try:
        img_plot = Image.open(BytesIO(response.content)).resize(IMAGE_SIZE)
    except:
        pic_malfunction.append(listings_meta.iloc[i]["id"])
        continue
    
    # save rgb data
    rgb_data = np.array(img_plot)
    save_path = "/content/drive/MyDrive/Colab/airbnb/munich/thumbnails/thumbnails_raw/hostpic" + str(listings_meta.iloc[i]["id"])
    np.save(save_path, rgb_data)